# CAR DEKHO

In [1]:
import pandas as pd
df=pd.read_csv(r"C:\Users\ADMIN\Documents\cardekho.csv")
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.2,5.0


In [10]:
df.isnull().sum()

year                  0
selling_price         0
km_driven             0
fuel                  0
owner                 0
mileage(km/ltr/kg)    0
engine                0
max_power             0
dtype: int64

In [3]:
df["mileage(km/ltr/kg)"]=df["mileage(km/ltr/kg)"].fillna(df["mileage(km/ltr/kg)"].mean())

In [4]:
df["engine"]=df["engine"].fillna(df["engine"].mean())

In [5]:
df['max_power'] = pd.to_numeric(df['max_power'], errors='coerce')

df["max_power"]=df["max_power"].fillna(df["max_power"].mean())

In [6]:
df.fillna({"seats": df["seats"].mode()[0]}, inplace=True)


In [7]:
df.drop(columns = ['name','seller_type','transmission','seats'],inplace=True)

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
encoder = LabelEncoder()

df['fuel'] = encoder.fit_transform(df['fuel'])


df['owner'] = encoder.fit_transform(df['owner'])


In [11]:
from sklearn.model_selection import train_test_split

In [13]:
x=df[["year","km_driven","fuel","owner","mileage(km/ltr/kg)","engine","max_power"]]
y=df["selling_price"]

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [36]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train_trf = scaler.fit_transform(x_train)
x_test_trf = scaler.transform(x_test)

In [37]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [38]:
model = Sequential()

model.add(Dense(64,activation='sigmoid',input_dim=10))
model.add(Dense(32,activation='relu'))
Dropout(0.2)
model.add(Dense(32,activation='relu'))
Dropout(0.2)
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

C:\Users\ADMIN\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 64)                  │             704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,929 (19.25 KB)

 Trainable params: 4,929 (19.25 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [41]:
# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=5,          # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restore model weights from the best epoch
)

In [48]:
history = model.fit(x_train_trf,y_train,batch_size=20,epochs=50,verbose=1,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 201298804736.0000 - mae: 283374.6250 - val_loss: 159438995456.0000 - val_mae: 262203.5625
Epoch 2/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 199914176512.0000 - mae: 283403.0938 - val_loss: 158338793472.0000 - val_mae: 261244.9219
Epoch 3/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 198630735872.0000 - mae: 282878.8438 - val_loss: 157270900736.0000 - val_mae: 260185.1094
Epoch 4/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 197324324864.0000 - mae: 280989.1562 - val_loss: 156219326464.0000 - val_mae: 259628.0938
Epoch 5/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 196079927296.0000 - mae: 281140.2812 - val_loss: 155174944768.0000 - val_mae: 259055.9375
Epoch 6/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 194846146560.0000 - mae: 280422.1562 - val_loss: 154166493184.0000 - val_mae: 257889.1875
Epoch 7/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 193621655552.0000 - mae: 279536.9062 - v

In [44]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(7,)))  # 👈 change input_shape to 7
# Add more layers as needed
model.add(Dense(units=1))  # Output layer

In [46]:
print(x_train_trf.shape)  # Should be (num_samples, 10)

(6502, 7)


In [49]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(7,)))  # adjust input_shape to match your data
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1))  # output layer

# ✅ Compile the model
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

# ✅ Train the model
history = model.fit(x_train_trf, y_train, batch_size=20, epochs=50, verbose=1,
                    validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0000e+00 - loss: -9317724.0000 - val_accuracy: 0.0000e+00 - val_loss: -9749261.0000
Epoch 2/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0000e+00 - loss: -10289867.0000 - val_accuracy: 0.0000e+00 - val_loss: -9749261.0000
Epoch 3/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0000e+00 - loss: -10289868.0000 - val_accuracy: 0.0000e+00 - val_loss: -9749261.0000
Epoch 4/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: -10289867.0000 - val_accuracy: 0.0000e+00 - val_loss: -9749261.0000
Epoch 5/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: -10289872.0000 - val_accuracy: 0.0000e+00 - val_loss: -9749261.0000
Epoch 6/50
261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0000e+00 - loss: -10289863.0000 - val_accuracy: 0.0000e+00 - val_loss: -9749261.0000
